In [30]:
import os
from os.path import join
import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
import trading as trd

In [32]:
# relaod module after modifying it
import importlib
importlib.reload(trd.stock_history)
importlib.reload(trd.portfolio)
importlib.reload(trd.benchmarks)
importlib.reload(trd.rl)
importlib.reload(trd.nn)
importlib.reload(trd)

<module 'trading' from 'C:\\Users\\hamza\\Documents\\code\\python\\cs229\\src\\trading\\__init__.py'>

Recompute all the perfomance metrics becaue of an off-by-one error

Evaluating model performance puts a zero in the last entry of a portfolio, resulting in a massive losss

In [33]:
model_folders = glob.glob(trd.MODEL_LOC + '\\*')

In [36]:
for model_folder in model_folders:
    q_perf = []
    nothing_perf = []
    rebal_perf = []

    with open(join(model_folder, 'test.csv')) as file:
        for line in file:
            (L, H) = line.strip().split(',')
            port_name = '{:s}_{:s}'.format(L, H)

            rebal = pd.read_csv(join(model_folder, trd.REBAL_DIR, port_name + '.csv'), index_col=0)
            nothing = pd.read_csv(join(model_folder, trd.NOTHING_DIR, port_name + '.csv'), index_col=0)
            q = pd.read_csv(join(model_folder, trd.Q_PORT_DIR, port_name + '.csv'), index_col=0)
            q = q.dropna()
            # theres a weird edge case with the last value of the q ports not being calculated
            q = q[:-1]
            
            q_perf.append(trd.portfolio_metrics(q, trd.sharpe_ratio_reward))
            nothing_perf.append(trd.portfolio_metrics(nothing, trd.sharpe_ratio_reward))
            rebal_perf.append(trd.portfolio_metrics(rebal, trd.sharpe_ratio_reward))

        q_perf = np.mean(np.asarray(q_perf), axis=0)
        nothing_perf = np.mean(np.asarray(nothing_perf), axis=0)
        rebal_perf = np.mean(np.asarray(rebal_perf), axis=0)

        np.savetxt(join(model_folder, 'nothing_perf.csv'), nothing_perf)
        np.savetxt(join(model_folder, 'rebal_perf.csv'), rebal_perf)
        np.savetxt(join(model_folder, 'q_perf.csv'), q_perf)
    
    print(model_folder)